# Burnout in the Tech Industry

Burnout is the result of excessive and prolonged stress in the workplace. It impacts you physically, emotionally, and mentally. It has a negative impact on work performance as well.

Here is a short video about burnout.

After working as a teacher for 8 years, I saw many colleagues struggle with burnout and some leave the profession as a result. I wondered was this a concern for my new career and decided to investigate.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

Initially I considered using multiple databases. However, after a quick EDA I realized two of the datasets would now work. Below is one of the datasets I decided not to use. While the column `Daily_Shouting` is intriguing, there was no connection to work, so I decided to save this for another time. 

In [2]:
survey_df = pd.read_csv('data/Wellbeing_and_lifestyle_data.csv')

In [3]:
survey_df.head()

,Timestamp,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,...,LIVE_VISION,SLEEP_HOURS,LOST_VACATION,DAILY_SHOUTING,SUFFICIENT_INCOME,PERSONAL_AWARDS,TIME_FOR_PASSION,DAILY_MEDITATION,AGE,GENDER
0,7/7/15,3,2,2,5,0,5,2,0,1,...,0,7,5,5,1,4,0,5,36 to 50,Female
1,7/7/15,2,3,4,3,8,10,5,2,2,...,5,8,2,2,2,3,2,6,36 to 50,Female
2,7/7/15,2,3,3,4,4,10,3,2,2,...,5,8,10,2,2,4,8,3,36 to 50,Female
3,7/7/15,3,3,10,3,10,7,2,5,2,...,0,5,7,5,1,5,2,0,51 or more,Female
4,7/7/15,5,1,3,3,10,4,2,4,2,...,0,7,0,0,2,8,1,5,51 or more,Female


In [4]:
survey_df.describe()

,FRUITS_VEGGIES,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,TODO_COMPLETED,FLOW,DAILY_STEPS,LIVE_VISION,SLEEP_HOURS,LOST_VACATION,DAILY_SHOUTING,SUFFICIENT_INCOME,PERSONAL_AWARDS,TIME_FOR_PASSION,DAILY_MEDITATION
count,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000,12756.000000
mean,2.930229,5.338899,5.484635,5.577767,6.551427,3.963311,2.700925,1.400753,5.706256,3.126764,5.704845,3.711979,7.035121,2.833098,2.920508,1.728206,5.702963,3.266855,6.252822
std,1.451959,3.314035,2.851588,3.262486,3.083920,2.752040,1.863933,0.490070,2.633266,2.334881,2.885340,3.244291,1.202189,3.672006,2.689751,0.444902,3.108637,2.721427,3.034980
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,2.000000,3.000000,3.000000,3.000000,4.000000,2.000000,1.000000,1.000000,4.000000,1.000000,3.000000,1.000000,6.000000,0.000000,1.000000,1.000000,3.000000,1.000000,4.000000
50%,3.000000,5.000000,5.000000,5.000000,6.000000,3.000000,3.000000,1.000000,6.000000,3.000000,5.000000,3.000000,7.000000,0.000000,2.000000,2.000000,5.000000,2.000000,7.000000
75%,4.000000,8.000000,8.000000,10.000000,10.000000,6.000000,5.000000,2.000000,8.000000,4.000000,8.000000,5.000000,8.000000,5.000000,4.000000,2.000000,9.000000,5.000000,10.000000
max,5.000000,10.000000,10.000000,10.000000,10.000000,10.000000,5.000000,2.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,2.000000,10.000000,10.000000,10.000000


In [5]:
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [6]:
survey_df.tail()

,Timestamp,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,TODO_COMPLETED,FLOW,DAILY_STEPS,LIVE_VISION,SLEEP_HOURS,LOST_VACATION,DAILY_SHOUTING,SUFFICIENT_INCOME,PERSONAL_AWARDS,TIME_FOR_PASSION,DAILY_MEDITATION,AGE,GENDER
12751,2/23/2020 22:03:56,3,4,10,8,10,8,6,5,1,7,4,3,3,7,0,1,1,10,6,7,21 to 35,Female
12752,2/24/2020 2:44:30,3,3,6,5,2,5,1,0,2,0,0,6,0,7,0,0,2,3,0,2,36 to 50,Female
12753,2/24/2020 10:35:02,4,4,7,5,3,3,4,2,1,7,3,4,5,6,0,2,2,6,3,5,36 to 50,Female
12754,2/24/2020 12:54:10,3,3,10,4,8,10,3,3,1,10,3,2,1,6,0,1,1,10,1,10,21 to 35,Female
12755,2/24/2020 12:55:21,2,2,6,1,6,1,5,2,1,4,3,7,0,7,0,1,2,3,2,10,51 or more,Female


I decided to focus on a Survey that had been conducted from 2014 - 2019. Here, participants in the tech indutry answered 118 questions about themselves and their job.

In [7]:
con = sqlite3.connect('data/mental_health.sqlite')
survey_year_df = pd.read_sql_query("SELECT * from Survey", con)

In [8]:
survey_year_df.head()

,SurveyID,Description
0,2014,mental health survey for 2014
1,2016,mental health survey for 2016
2,2017,mental health survey for 2017
3,2018,mental health survey for 2018
4,2019,mental health survey for 2019


In [9]:
question_df = pd.read_sql_query("SELECT * from Question", con)

In [10]:
question_df.loc[3]['questiontext']

'If you live in the United States, which state or territory do you live in?'

In [11]:
question_df.loc[103]['questiontext']

'Which of the following best describes your work position?'

I wanted to be able to get an idea of the questions asked. This way I could narrow down what I wanted to focus on.

In [12]:
question_df

,questiontext,questionid
0,What is your age?,1
1,What is your gender?,2
2,What country do you live in?,3
3,"If you live in the United States, which state ...",4
4,Are you self-employed?,5
5,Do you have a family history of mental illness?,6
6,Have you ever sought treatment for a mental he...,7
7,How many employees does your company or organi...,8
8,Is your employer primarily a tech company/orga...,9
9,Does your employer provide mental health benef...,10


In [13]:
answer_df = pd.read_sql_query("SELECT * from Answer", con)

In [14]:
answer_df[answer_df['QuestionID'] == 75].head()

,AnswerText,SurveyID,UserID,QuestionID
160557,-1,2017,2694,75
160558,-1,2017,2695,75
160559,-1,2017,2696,75
160560,-1,2017,2697,75
160561,They talked about as caregivers of family member.,2017,2698,75


In [33]:
answer_df[answer_df['UserID'] == 2694].shape
answer_df[answer_df['UserID'] == 2694]

,AnswerText,SurveyID,UserID,QuestionID
114441,27,2017,2694,1
115197,Female,2017,2694,2
115953,United Kingdom,2017,2694,3
116709,-1,2017,2694,4
117465,0,2017,2694,5
118221,No,2017,2694,6
118977,1,2017,2694,7
119733,100-500,2017,2694,8
120489,1,2017,2694,9
121245,No,2017,2694,10


I wanted to merge the two dataframes into one. However, first I needed to rename the columns in one of the dataframes for consistency and a proper merge.

In [40]:
question_df = question_df.rename(columns={"questiontext": "QuestionText", "questionid": "QuestionID"})

In [63]:
merged_df = question_df.merge(right = answer_df, how = 'outer', on = 'QuestionID')
merged_df.head()

,QuestionText,QuestionID,AnswerText,SurveyID,UserID
0,What is your age?,1,37,2014,1
1,What is your age?,1,44,2014,2
2,What is your age?,1,32,2014,3
3,What is your age?,1,31,2014,4
4,What is your age?,1,31,2014,5


Then I decided to reorganize the data frame to better view the information.

In [65]:
pivoted_df = pd.pivot_table(merged_df, values = 'AnswerText', index = ['SurveyID', 'UserID'], columns = ['QuestionText'], aggfunc = 'first')
pivoted_df.tail()

QuestionText    Any additional notes or comments  \
SurveyID UserID                                    
2019     4214                                NaN   
         4215                                NaN   
         4216                                NaN   
         4217                                NaN   
         4218                                NaN   

QuestionText    Are you openly identified at work as a person with a mental health issue?  \
SurveyID UserID                                                                             
2019     4214                                                    0                          
         4215                                                    0                          
         4216                                                    0                          
         4217                                                    0                          
         4218                                                    1                          

QuestionText    Are you self-employed?  \
SurveyID UserID                          
2019     4214                        0   
         4215                        1   
         4216                        1   
         4217                        0   
         4218                        0   

QuestionText    Briefly describe what you think the industry as a whole and/or employers could do to improve mental health support for employees.  \
SurveyID UserID                                                                                                                                     
2019     4214                                                   -1                                                                                  
         4215                                                  IDK                                                                                  
         4216                                         sdsdsdsdsdsd                                                                                  
         4217                                      raise awareness                                                                                  
         4218    reduce stigma. offer options for part time wor...                                                                                  

QuestionText    Describe the circumstances of the badly handled or unsupportive response.  \
SurveyID UserID                                                                             
2019     4214                                                   -1                          
         4215    employee was ostracized and employer discussed...                          
         4216                                                   -1                          
         4217                                                   -1                          
         4218                                                   -1                          

QuestionText    Describe the circumstances of the supportive or well handled response.  \
SurveyID UserID                                                                          
2019     4214                                                   -1                       
         4215                                                   -1                       
         4216                                                   -1                       
         4217                                                   -1                       
         4218                                                   -1                       

QuestionText    Describe the conversation with coworkers you had about your mental health including their reactions.  \
SurveyID UserID                                                                                                        
2019     4214                                                   -1                                                     
 

Then I decided to take a look at characteristics of the participants that completed the survey from over the 6 years.

I want to get a count of `Gender`, `Age`, `Race`, and other descriptors.

In [21]:
Males = (answer_df.AnswerText == 'Male').sum() + (answer_df.AnswerText == 'male').sum()
Males

3042

In [18]:
Females = (answer_df.AnswerText == 'Female').sum() + (answer_df.AnswerText == 'female').sum()
Females

1024

In [19]:
Nonbinary = (answer_df.AnswerText == 'Nonbinary').sum() + (answer_df.AnswerText == 'nonbinary').sum()
Nonbinary

9